# Personal ChatBot Assistant

Here I built a tool that takes a question and responds with an explanation.

Is it possible selecting whether using a local running model (llama 3.2) or calling OpenAI APIs (4o-mini).

Using the Gradio UI is also possible making the tool more user-friendly.

In [1]:
# imports
from dotenv import load_dotenv
import os
from openai import OpenAI
import ollama
from IPython.display import Markdown, display
import gradio as gr
import anthropic

In [2]:
# Constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
MODEL_ANTHROPIC = 'claude-3-haiku-20240307'

In [3]:
# Set up environment
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
if openai_api_key:
    print("OpenAI API Key found")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print("Anthropic API Key found")
else:
    print("Anthropic API Key not set")


openai = OpenAI()
claude = anthropic.Anthropic()

OpenAI API Key not set
Anthropic API Key not set


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [27]:
# This function returns the messages template
def get_messages(system_prompt, user_prompt):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [60]:
# This function calls OpenAI API
def openai_stream(system_prompt, user_prompt):   
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=get_messages(system_prompt, user_prompt),
        stream=True
    )
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

In [61]:
def claude_stream(system_prompt, user_prompt):
    result = claude.messages.stream(
        model=MODEL_ANTHROPIC,
        max_tokens=1000,
        temperature=0.7,
        system=system_prompt,
        messages=[
            {"role": "user", "content": user_prompt}
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [62]:
# This function queries the local running model
def ollama_stream(system_prompt, user_prompt):
    stream = ollama.chat(model=MODEL_LLAMA, messages=get_messages(system_prompt, user_prompt), stream=True)
    response = ""
    for chunk in stream:
        response += chunk['message']['content'] or ""
        yield response

In [65]:
def model_stream(user_prompt, model):
    system_prompt = "You are a powerful assistant that takes questions and responds with an explanation. You are provided with questions from a user. Respond in markdown."
    if model == "GPT":
        result = openai_stream(system_prompt, user_prompt)
    elif model == "Claude":
        result = claude_stream(system_prompt, user_prompt)
    else:
        result = ollama_stream(system_prompt, user_prompt)
    yield from result

In [66]:
view = gr.Interface(
    fn=model_stream,
    inputs=[gr.Textbox(label="Your Request"), gr.Dropdown(["GPT", "Claude", "LLaMA (local run)"], label="Select model", value="LLaMA (local run)")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode='never'
)
view.launch()

* Running on local URL:  http://127.0.0.1:7891

To create a public link, set `share=True` in `launch()`.
